This is the Code for Dead. It did not go well. The idea was to use the architecture from Bernard but whilst calculating the losses and training the discriminator every 50 epochs.

We also changed the generator to be a reverse VGG with the Batch Normalization after every Conv2D layer (ResNet)

AS well as this we added some Data Augmentation to the discriminator. 

In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten,BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import os
from gandywarhol.data_gen import CustomDataGen


In [20]:
def build_discriminator(image_shape):
    model = Sequential()
    model.add(layers.RandomFlip('horizontal_and_vertical'))
    model.add(layers.RandomRotation(0.2))
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [22]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 512, activation='relu',       input_dim=noise_size))
    model.add(Reshape((4, 4, 512)))
    model.add(UpSampling2D())
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization(epsilon=1.001e-5))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    model.summary()
    input_ = Input(shape=(noise_size,))
    generated_image = model(input_)
    
    return Model(input_, generated_image)

In [25]:
BATCH_SIZE = 32
path_list = ['../raw_data/'+image for image in os.listdir('../raw_data') if image.endswith('jpeg')]



In [29]:
path_list

['../raw_data/impress1.jpeg',
 '../raw_data/impress7.jpeg',
 '../raw_data/impress6.jpeg',
 '../raw_data/impress5.jpeg',
 '../raw_data/impress4.jpeg',
 '../raw_data/impress8.jpeg',
 '../raw_data/impress3.jpeg',
 '../raw_data/impress2.jpeg']

In [26]:
train_set = CustomDataGen(path_list)

In [27]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [28]:
def save_images(cnt, noise, epoch):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
    output_path = 'thurs_resnet_imgs'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"img_epoch-{epoch}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [ ]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = Adam(learning_rate = 1.5e-4)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])

generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)

# could you compile the generator without the discriminator?
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])

In [ ]:
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
discriminator.trainable = False

fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1
for epoch in tqdm(range(EPOCHS)):
    for index in tqdm(range(len(train_set))):
        x_real = train_set[index]/255.
        noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
        x_fake = generator.predict(noise)
        if epoch % 50 == 0 and index<10:
            discriminator.trainable = True
    
        else:
            discriminator.trainable = False
        discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
        discriminator_metric_generated = discriminator.train_on_batch(
        x_fake, y_fake)
        discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
        discriminator.trainable = False
        generator_metric = combined.train_on_batch(noise, y_real)
    with open('losses.txt', 'a') as f:
        f.write(f'epoch: {epoch} disc loss: {discriminator_metric[0]}, gen loss: {generator_metric[0]}.\n')
    train_set.on_epoch_end()
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise, epoch)
        # save_model(epoch, model)
        cnt += 1
        print(f'{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}')